In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/intents/intents.json


In [3]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

2024-02-12 09:33:26.417503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 09:33:26.417740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 09:33:26.579850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
with open('/kaggle/input/intents/intents.json') as file:
    data = json.load(file)
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
training_labels

array([ 9,  9,  9,  9,  9,  8,  8,  8, 16, 16, 16, 16,  0,  0,  0, 11, 11,
       11, 10, 10, 10, 10, 10, 10, 10,  5,  5,  5,  5,  5,  3,  3,  3, 12,
       12, 12, 12, 12,  2,  2,  2, 15, 15, 15, 15, 13, 13, 13, 13, 14, 14,
       14, 14,  1,  1,  1,  6,  6,  6,  4,  4,  4,  7,  7,  7])

In [10]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = 'OOV'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
word_index

{'OOV': 1,
 'i': 2,
 'you': 3,
 'a': 4,
 'can': 5,
 'to': 6,
 'how': 7,
 'order': 8,
 'what': 9,
 'me': 10,
 'account': 11,
 'an': 12,
 'is': 13,
 'for': 14,
 'help': 15,
 'are': 16,
 'do': 17,
 'need': 18,
 'my': 19,
 'there': 20,
 'support': 21,
 'create': 22,
 'new': 23,
 'want': 24,
 'complaint': 25,
 'cancel': 26,
 'payment': 27,
 'return': 28,
 'product': 29,
 'contact': 30,
 'feedback': 31,
 'thanks': 32,
 'who': 33,
 'your': 34,
 'name': 35,
 'give': 36,
 'please': 37,
 'open': 38,
 'have': 39,
 'service': 40,
 'process': 41,
 'shipping': 42,
 'methods': 43,
 'this': 44,
 'available': 45,
 'discounts': 46,
 'customer': 47,
 'hi': 48,
 'hey': 49,
 'anyone': 50,
 'hello': 51,
 'hay': 52,
 'bye': 53,
 'see': 54,
 'later': 55,
 'goodbye': 56,
 'thank': 57,
 "that's": 58,
 'helpful': 59,
 'the': 60,
 'should': 61,
 'call': 62,
 'whats': 63,
 'could': 64,
 'hand': 65,
 'raise': 66,
 'about': 67,
 'place': 68,
 'buy': 69,
 'something': 70,
 'from': 71,
 'here': 72,
 'purchase': 73,
 '

In [11]:
seq = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(seq, truncating='post', maxlen=max_len)
seq

[[48],
 [49],
 [13, 50, 20],
 [51],
 [52],
 [53],
 [54, 3, 55],
 [56],
 [32],
 [57, 3],
 [58, 59],
 [32, 14, 60, 15],
 [33, 16, 3],
 [9, 16, 3],
 [33, 3, 16],
 [9, 13, 34, 35],
 [9, 61, 2, 62, 3],
 [63, 34, 35],
 [64, 3, 15, 10],
 [36, 10, 4, 65, 37],
 [5, 3, 15],
 [9, 5, 3, 17, 14, 10],
 [2, 18, 4, 21],
 [2, 18, 4, 15],
 [21, 10, 37],
 [2, 18, 6, 22, 4, 23, 11],
 [7, 6, 38, 4, 23, 11],
 [2, 24, 6, 22, 12, 11],
 [5, 3, 22, 12, 11, 14, 10],
 [7, 6, 38, 4, 23, 11],
 [39, 4, 25],
 [2, 24, 6, 66, 4, 25],
 [20, 13, 4, 25, 67, 4, 40],
 [7, 5, 2, 68, 12, 8],
 [2, 24, 6, 69, 70],
 [5, 2, 8, 71, 72],
 [7, 6, 73],
 [74, 41],
 [2, 18, 6, 26, 19, 8],
 [5, 2, 26, 19, 8],
 [7, 6, 26, 12, 8],
 [7, 75, 76, 42, 77],
 [78, 79, 2, 80, 19, 8],
 [42, 81],
 [82, 83],
 [9, 27, 43, 17, 3, 84],
 [7, 5, 2, 85, 14, 19, 8],
 [27, 86],
 [87, 27, 43],
 [7, 17, 2, 28, 12, 88],
 [5, 2, 28, 4, 29],
 [28, 41],
 [89, 90],
 [13, 44, 29, 45],
 [17, 3, 39, 44, 91, 92],
 [29, 93],
 [16, 20, 94, 46, 45],
 [5, 2, 95, 4, 96],


In [12]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 17)                289       
                                                                 
Total params: 16833 (65.75 KB)
Trainable params: 16833 (65.75 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [14]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
3/3 [==============================] - 1s 7ms/step - loss: 2.8349 - accuracy: 0.0462
Epoch 2/500
3/3 [==============================] - 0s 5ms/step - loss: 2.8335 - accuracy: 0.0462
Epoch 3/500
3/3 [==============================] - 0s 4ms/step - loss: 2.8327 - accuracy: 0.0462
Epoch 4/500
3/3 [==============================] - 0s 5ms/step - loss: 2.8320 - accuracy: 0.0462
Epoch 5/500
3/3 [==============================] - 0s 5ms/step - loss: 2.8313 - accuracy: 0.0462
Epoch 6/500
3/3 [==============================] - 0s 4ms/step - loss: 2.8307 - accuracy: 0.0462
Epoch 7/500
3/3 [==============================] - 0s 5ms/step - loss: 2.8303 - accuracy: 0.0462
Epoch 8/500
3/3 [==============================] - 0s 4ms/step - loss: 2.8298 - accuracy: 0.0462
Epoch 9/500
3/3 [==============================] - 0s 4ms/step - loss: 2.8292 - accuracy: 0.0462
Epoch 10/500
3/3 [==============================] - 0s 4ms/step - loss: 2.8286 - accuracy: 0.0462
Epoch 11/500
3/3 [===========

model.save('chatbot_model')

import 

In [17]:
model.save('chatbot_model')
import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("/kaggle/input/intents/intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('/kaggle/working/chatbot_model')

    # load tokenizer object
    with open('/kaggle/working/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('/kaggle/working/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()